<a href="https://colab.research.google.com/github/Feterlike/STG/blob/main/Satellite_imagery_video_timelapse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌍 Satellite Timelapse Generator

### Platform Behavior Note:
- **Runtime Restarts**: If you restart the runtime, the app and the public link will stop working immediately.
- **Closing Tab**: If you close the Google Colab tab, the runtime will eventually time out and kill the app.
- **Stopping Cell**: If you manually stop the code cell, the app will shut down.

### How to use:
1. Run the **Installation** cell below.
2. Run the **Run App** cell to start the web interface.
3. **Click the Public URL** to open the app in a new tab.

In [ ]:
# @title 🛠️ Installation
!pip install --quiet opencv-python requests geopy pillow gradio

In [ ]:
# @title 🚀 Run App
import os
import math
import cv2
import requests
import shutil
import numpy as np
import gradio as gr
import signal
from datetime import date, timedelta
from concurrent.futures import ThreadPoolExecutor
from geopy.geocoders import Nominatim
from PIL import Image, ImageDraw, ImageFont

# Pre-launch Cleanup: Close any existing app instances in this kernel
try:
    if 'demo' in locals():
        demo.close()
except:
    pass

# --- CONFIGURATION ---
MAX_DIM = 1000

def get_tile_coords(lat, lon, zoom):
    tile_width_deg = 288.0 / (2 ** zoom)
    col = math.floor((lon + 180) / tile_width_deg)
    row = math.floor((90 - lat) / tile_width_deg)
    return row, col

def get_pixel_coords(lat, lon, row, col, zoom, tile_size=512):
    tile_width_deg = 288.0 / (2 ** zoom)
    tile_lon_start = (col * tile_width_deg) - 180
    tile_lat_start = 90 - (row * tile_width_deg)
    lon_offset = lon - tile_lon_start
    lat_offset = tile_lat_start - lat
    x = int((lon_offset / tile_width_deg) * tile_size)
    y = int((lat_offset / tile_width_deg) * tile_size)
    return x, y

def add_overlays(cv_img, label, date_text, pin_coords=None):
    img_pil = Image.fromarray(cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    try: font = ImageFont.truetype("DejaVuSans-Bold.ttf", 20)
    except IOError: font = ImageFont.load_default()
    bbox = draw.textbbox((0, 0), date_text, font=font)
    text_w, text_h = bbox[2] - bbox[0], bbox[3] - bbox[1]
    w, h = img_pil.size
    draw.text((w - text_w - 10, h - text_h - 10), date_text, font=font, fill=(255, 255, 255), stroke_width=2, stroke_fill=(0,0,0))
    if pin_coords:
        px, py = pin_coords
        radius = 5
        draw.ellipse((px - radius, py - radius, px + radius, py + radius), fill=(255, 0, 0), outline=(0, 0, 0))
        bbox = draw.textbbox((0, 0), label, font=font)
        cw, ch = bbox[2] - bbox[0], bbox[3] - bbox[1]
        cx, cy = px - cw // 2, py - radius - ch - 5
        cx, cy = max(5, min(w - cw - 5, cx)), max(5, min(h - ch - 5, cy))
        draw.text((cx, cy), label, font=font, fill=(255, 255, 255), stroke_width=2, stroke_fill=(0,0,0))
    elif label: draw.text((10, 10), label, font=font, fill=(255, 255, 255), stroke_width=2, stroke_fill=(0,0,0))
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def download_tile(date_str, row, col, zoom, path):
    url = (f"https://gibs.earthdata.nasa.gov/wmts/epsg4326/best/MODIS_Terra_CorrectedReflectance_TrueColor/default/{date_str}/250m/{zoom}/{row}/{col}.jpg")
    if os.path.exists(path): return True
    try:
        r = requests.get(url, timeout=5)
        if r.status_code == 200:
            with open(path, 'wb') as f: f.write(r.content)
            return True
    except: pass
    return False

def generate_timelapse(input_mode, city, lat, lon, b_lat1, b_lon1, b_lat2, b_lon2, year, zoom, fps, progress=gr.Progress()):
    geolocator = Nominatim(user_agent="sat_timelapse_gen")
    pin_coords_logic = None; label = ""
    if input_mode == "City":
        loc = geolocator.geocode(city)
        if not loc: return f"City not found.", None
        u_lat, u_lon = loc.latitude, loc.longitude; label = city; pin_coords_logic = (u_lat, u_lon)
        rs, cs = get_tile_coords(u_lat, u_lon, zoom); re, ce = rs, cs
    elif input_mode == "Coordinate":
        label = f"Lat: {lat}, Lon: {lon}"; pin_coords_logic = (lat, lon)
        rs, cs = get_tile_coords(lat, lon, zoom); re, ce = rs, cs
    else:
        r1, c1 = get_tile_coords(b_lat1, b_lon1, zoom); r2, c2 = get_tile_coords(b_lat2, b_lon2, zoom)
        rs, re, cs, ce = min(r1, r2), max(r1, r2), min(c1, c2), max(c1, c2)
        if (re-rs+1)*(ce-cs+1) > 25: return "Area too large.", None

    folder = f"temp_proj_{year}"
    if os.path.exists(folder): shutil.rmtree(folder)
    os.makedirs(folder, exist_ok=True)
    s_date, e_date = date(year, 1, 1), date(year, 12, 31)
    if year == date.today().year: e_date = date.today() - timedelta(days=1)
    all_d = [(s_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((e_date - s_date).days + 1)]

    final_imgs = []
    for idx, d_str in enumerate(all_d):
        df = os.path.join(folder, d_str); os.makedirs(df, exist_ok=True)
        tasks = [(d_str, r, c, zoom, os.path.join(df, f"{r}_{c}.jpg")) for r in range(rs, re+1) for c in range(cs, ce+1)]
        with ThreadPoolExecutor(max_workers=10) as ex: ex.map(lambda t: download_tile(*t), tasks)
        ch, cw = (re-rs+1)*512, (ce-cs+1)*512; canvas = np.zeros((ch, cw, 3), dtype=np.uint8); valid = False
        for r in range(rs, re+1):
            for c in range(cs, ce+1):
                p = os.path.join(df, f"{r}_{c}.jpg")
                if os.path.exists(p):
                    img = cv2.imread(p)
                    if img is not None: canvas[(r-rs)*512:(r-rs)*512+512, (c-cs)*512:(c-cs)*512+512] = img; valid = True
        if valid:
            scale = min(MAX_DIM/cw, MAX_DIM/ch); resized = cv2.resize(canvas, (int(cw*scale), int(ch*scale)))
            px = None
            if pin_coords_logic:
                tx, ty = get_pixel_coords(pin_coords_logic[0], pin_coords_logic[1], rs, cs, zoom)
                px = (int(tx*scale), int(ty*scale))
            ff = add_overlays(resized, label, d_str, pin_coords=px); out_p = os.path.join(folder, f"f_{idx:03d}.jpg"); cv2.imwrite(out_p, ff); final_imgs.append(out_p)
        shutil.rmtree(df); progress((idx+1)/len(all_d)*0.8, desc=f"Processing {d_str}...")

    vn = f"timelapse_{year}.mp4"; h, w, _ = cv2.imread(final_imgs[0]).shape
    out = cv2.VideoWriter(vn, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
    for p in final_imgs: out.write(cv2.imread(p))
    out.release(); f_v = f"final_timelapse_{year}.mp4"
    os.system(f'ffmpeg -y -i "{vn}" -vcodec libx264 -crf 28 -preset fast -loglevel error "{f_v}"')
    shutil.rmtree(folder);
    if os.path.exists(vn): os.remove(vn)
    return "Success!", f_v

def stop_app():
    os.kill(os.getpid(), signal.SIGTERM)
    return "Stopping..."

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🌍 Advanced Satellite Timelapse Generator")
    with gr.Row():
        with gr.Column():
            mode = gr.Radio(["City", "Coordinate", "Bounding Box"], label="Mode", value="City")
            with gr.Group() as g1: city_in = gr.Textbox(label="City")
            with gr.Group(visible=False) as g2: lat_in = gr.Number(label="Lat"); lon_in = gr.Number(label="Lon")
            with gr.Group(visible=False) as g3:
                gr.Markdown("Bounding Box")
                with gr.Row(): b_lat1 = gr.Number(label="Lat1", value=40.0); b_lon1 = gr.Number(label="Lon1", value=-74.0)
                with gr.Row(): b_lat2 = gr.Number(label="Lat2", value=39.0); b_lon2 = gr.Number(label="Lon2", value=-73.0)
            yr = gr.Number(label="Year", value=2024, precision=0)
            zm = gr.Slider(0, 9, step=1, label="Zoom", value=6); fs = gr.Slider(1, 60, step=1, label="FPS", value=15)
            btn = gr.Button("🚀 Generate", variant="primary")
        with gr.Column():
            st = gr.Markdown(); vi = gr.Video(label="Video"); stop = gr.Button("🛑 Stop App", variant="stop")

    mode.change(lambda m: {g1: gr.update(visible=(m=="City")), g2: gr.update(visible=(m=="Coordinate")), g3: gr.update(visible=(m=="Bounding Box"))}, inputs=[mode], outputs=[g1, g2, g3])
    btn.click(fn=generate_timelapse, inputs=[mode, city_in, lat_in, lon_in, b_lat1, b_lon1, b_lat2, b_lon2, yr, zm, fs], outputs=[st, vi])
    stop.click(fn=stop_app, outputs=[st])

demo.launch(share=True, inline=False)